In [10]:
import preprocessor as pp
import numpy as np
import tensorflow as tf
#import tensorflow_addons as tfa
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [11]:
['5'] * np.zeros((2,3)).shape[0]

['5', '5']

In [2]:
def gen_output_files(ids, p_tests, p_sepsis, p_vitals, filename='output.csv'):
    header = 'pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate'
    output = np.concatenate((ids, p_tests, p_sepsis, p_vitals), axis=1)

    fmt = ['%d']
    for i in range(output.shape[1] - 1):
        fmt.append('%.3f')

    np.savetxt(filename, output, delimiter=',',
               fmt=fmt, header=header, comments='')

In [3]:
train_feature_file = 'train_features.csv'
train_label_file = 'train_labels.csv'
test_features_file = 'test_features.csv'
processor = pp.preprocessor(
    train_feature_file, train_label_file, test_features_file, normalize=False)

print('starting data parsing...')
train_ids, train_features = processor.get_train_features()
label_ids, label_tests, label_sepsis, label_vitals = processor.get_train_labels()
test_ids, test_features = processor.get_test_features()


starting data parsing...
Total:18995
    Selected: 18995
    Removed: 0 (0.00% of total)



In [4]:
train_features[3, :, 5]

array([4.05 , 4.05 , 4.05 , 3.855, 3.66 , 3.66 , 3.66 , 3.66 , 3.66 ,
       3.66 , 3.66 , 3.66 ])

In [2]:
'hi {}'.format('hello')

'hi hello'

In [4]:
sepsis_cases = []
for i in range(train_features.shape[0]):
    if label_sepsis[i] == 1:
        sepsis_cases.append(train_features[i])
sepsis_cases = np.stack(sepsis_cases)

balanced_train_features = train_features
balanced_label_sepsis = label_sepsis
for i in range(15):
    balanced_train_features = np.vstack(
            (balanced_train_features, sepsis_cases))
    balanced_label_sepsis = np.concatenate(
            (balanced_label_sepsis, np.ones(shape=(sepsis_cases.shape[0],))))
shuffle = np.arange(len(balanced_label_sepsis))
np.random.shuffle(shuffle)
balanced_train_features = balanced_train_features[shuffle]
balanced_label_sepsis = balanced_label_sepsis[shuffle]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [73]:
features_sepsis, featues_sepsis_val, label_sepsis_train, label_sepsis_val = train_test_split(train_features, label_sepsis, test_size=0.2)

In [90]:
featues_sepsis_val[[1,2,54,3455]].shape

(4, 12, 36)

In [78]:
label_sepsis_val.shape

(3799,)

In [86]:
a = np.arange(10,15)
n = np.arange(5)
a[[0,2]]

array([10, 12])

In [48]:
len(balanced_label_sepsis[balanced_label_sepsis == 1]) / len(balanced_label_sepsis)

0.4929350134503752

In [65]:
print(balanced_label_sepsis.shape)
print(balanced_train_features.shape)
print(train_features.shape)

(1, 35315)
(1, 35315, 12, 36)
(18995, 12, 36)


In [11]:
neg, pos = len(label_sepsis[label_sepsis == 0]), len(label_sepsis[label_sepsis == 1])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
initial_bias = np.log([pos/neg])

Examples:
    Total: 18995
    Positive: 1088 (5.73% of total)



In [6]:
train_features[0].shape

(12, 36)

In [8]:
label_sepsis[label_sepsis == 1]




array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
'''
    predictions for test ordered

    roc auc evaluation
'''
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_AUC', 
    verbose=1,
    patience=12,
    mode='max',
    restore_best_weights=True)



model_tests = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(432,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
])

model_tests.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=[tf.keras.metrics.AUC(multi_label=True, name='AUC')])

model_tests.fit(train_features, label_tests, epochs= 50 ,validation_split=0.1, batch_size=64, callbacks=early_stopping)
predict_tests = model_tests.predict(test_features)
print(roc_auc_score(label_tests, model_tests.predict(train_features)))

#.87

In [ ]:
roc_auc_score(label_tests, model_tests.predict(train_features))

In [ ]:
'''
    predictions for sepsis

    -roc auc evaluation

    -problems with imbalanced data
'''
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_AUC', 
    verbose=1,
    patience=4,
    mode='max',
    restore_best_weights=True)

output_bias = np.log(len(label_sepsis[label_sepsis == 1]) / len(label_sepsis[label_sepsis == 0]))


model_sepsis = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape= (432,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    #tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='softmax', bias_initializer=tf.keras.initializers.Constant(output_bias))
])

model_sepsis.compile(optimizer=tf.optimizers.Adam(),
                    loss=tf.keras.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC')])

model_sepsis.fit(train_features, label_sepsis, epochs=3, validation_split=0.2, batch_size=32, callbacks=early_stopping)
predict_sepsis = model_sepsis.predict(test_features)
print(roc_auc_score(label_sepsis, model_sepsis.predict(train_features)))

#AUC .64

In [ ]:
np.around(predict_sepsis, 3)[:20].T

In [ ]:
len(label_sepsis[label_sepsis == 1]) / len(label_sepsis[label_sepsis == 0])

In [ ]:
len(predict_sepsis[predict_sepsis >= 0.5]) / len(predict_sepsis)

In [ ]:
'''
    predictions for vital signs 
'''


from tensorflow.keras import backend as K
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

    return out


def R2(y , y_pred):
    return np.mean([0.5 + 0.5 * max(0,r2_score(y[:,i], y_pred[:,i])) for i in range(4)])


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_mse',
    verbose=1,
    patience=12,
    mode='min',
    restore_best_weights=True)

print('learning vitals')
model_vitals = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, activation='relu', input_shape=(432,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(4)
])

model_vitals.compile(optimizer='adam',
                     loss='mean_squared_error',
                     metrics=['mse'])

model_vitals.fit(train_features, label_vitals, epochs=50, callbacks=early_stopping, validation_split=0.15, batch_size=32)
predict_vitals = model_vitals.predict(test_features)
print(R2(label_vitals, model_vitals.predict(train_features)))

#.74 R2 Score

In [ ]:
R2(label_vitals, model_vitals.predict(train_features))

In [ ]:
'''
output to csv file
'''

gen_output_files(test_ids, predict_tests, predict_sepsis, predict_vitals)

# zip the output.csv file
!rm output.zip
!zip output.zip output.csv



train_feature_file = 'train_features.csv'
train_label_file = 'train_labels.csv'
test_features_file = 'test_features.csv'

train_features = np.genfromtxt(train_feature_file, delimiter=',', skip_header=1)
train_labels = np.genfromtxt(train_label_file, delimiter=',', skip_header=1)

test_features = np.genfromtxt(test_features_file, delimiter=',', skip_header=1)

test_ids = test_features[::12,0].reshape(-1,1)